# Create Wrapper Function for Externally Deployed DataRobot Model

This notebook demonstrates how to create a WML wrapper function for model deployed externally in a DataRobot environment.  If models are deployed in an external environment and the payload input/output format of that environment exactly matches that of Watson Machine Learning, then the environment can be setup as a Custom Service Provider in OpenScale (https://www.ibm.com/docs/en/cloud-paks/cp-data/3.5.0?topic=models-custom-ml-frameworks).  Alternatively, if the payload input/output format does not match the format of Watson Machine Learning deployments, then the solution is to create a proxy WML endpoint for the external deployment.  The proxy endpoint is a python function that converts the input/output payloads into the correct format and calls the existing deployment.  In addition the function checks whether the model usability status in OpenPages is set to "Allow" before calling the endpoint.  For this exercise, we have deployed the airline delay model in an external environment called Heroku.  The input/output format for this deployment resembles that of a DataRobot model.

### Load Configuration File

In [5]:
import os
#os.environ["CONFIG_FILE"] = "wrapper_config.json"
config_file = os.environ.get('CONFIG_FILE').replace("\"","")

In [6]:
import json
wrapper_config=json.load(open(f"/project_data/data_asset/{config_file}"))

In [7]:
# Credentials for external endpoint
external_url = wrapper_config['external_endpoint_info']['external_url']
username = wrapper_config['external_endpoint_info']['username']
api_key = wrapper_config['external_endpoint_info']['api_key']
# WML information
deployment_space = wrapper_config['function_info']['deployment_space']
function_name = wrapper_config['function_info']['function_name']
deployment_name = wrapper_config['function_info']['deployment_name']
# OpenPages Information
op_api_key = wrapper_config['openpages_info']['op_api_key']
model_status_field = wrapper_config['openpages_info']['model_status_field']
mrm_model_id = wrapper_config['openpages_info']['mrm_model_id']
# CPD Auth
cpd_username=wrapper_config['cpd_credentials']['cpd_username']
cpd_api_key=wrapper_config['cpd_credentials']['cpd_api_key']

### Sample Scoring Request to Externally Deployed Model

In [ ]:
import requests, urllib3, json, inspect, textwrap
urllib3.disable_warnings()

#external_url = 'https://airline-delay-202109-group0.herokuapp.com/predict'
#username = 'datarobot'
#api_key = 'apikey_for_datarobot'
#header = {'Content-Type': 'application/json'}
#sample_payload = [{"DAY":11,"DAY_OF_WEEK":7,"ORIGIN_AIRPORT":"ABQ","DESTINATION_AIRPORT":"DFW","DEPARTURE_DELAY":2,"TAXI_OUT":11,"DISTANCE":570},{"DAY":11,"DAY_OF_WEEK":7,"ORIGIN_AIRPORT":"ABQ","DESTINATION_AIRPORT":"DFW","DEPARTURE_DELAY":2,"TAXI_OUT":11,"DISTANCE":570}]

In [ ]:
#r = requests.post(external_url, headers=header,data=json.dumps(sample_payload),auth=(username, api_key), verify=False)

In [ ]:
#r.json()

### Define Payload Conversion Functions
Below we define the functions which will be used to convert the payload from DR into WML format (and vice versa).  This functions will then be used inside of the python wrapper function. 

In [ ]:
def wml_input_payload_conv(wml_payload):
    fields = wml_payload['input_data'][0]['fields']
    datarobot_payload = []
    for value in wml_payload['input_data'][0]['values']:
        datarobot_payload.append({fields[i]: v for i, v in enumerate(value)})
    return datarobot_payload

In [ ]:
def dr_to_wml_output_payload_conv(dr_output):
    preds = list([d['prediction'],[d['predictionValues'][0]['value'],d['predictionValues'][1]['value']]] for d in dr_output['data'])
    wml_output = {"predictions": [{"fields": ['prediction','probability'], "values": preds}]}
    return wml_output

## Get Function Names Programmatically and Convert Function Source Code to String Format

In [ ]:
input_converter_string = textwrap.dedent(inspect.getsource(wml_input_payload_conv))
output_converter_string = textwrap.dedent(inspect.getsource(dr_to_wml_output_payload_conv))

input_converter_name = wml_input_payload_conv.__name__
output_converter_name = dr_to_wml_output_payload_conv.__name__

In [ ]:
cpd_host = "https://zen-cpd-zen.ibmcloud-roks-hhlrn-6ccd7f378ae819553d37d5f2ee142bd6-0000.mex01.containers.appdomain.cloud"
openpages_url = 'https://openpages-openpages-1-zen.ibmcloud-roks-hhlrn-6ccd7f378ae819553d37d5f2ee142bd6-0000.mex01.containers.appdomain.cloud'

## Define Wrapper Function
Below we define the wrapper function which will 1) convert the input/output payload of the external endpoint to WML 2) check the usability status of the model in OpenPages and 3) call the external endpoint if usability status is "Allow"

In [ ]:

params = {'ml_endpoint': external_url,'cpd_username': cpd_username,'cpd_api_key': cpd_api_key,'cpd_host': cpd_host,'openpages_url':openpages_url,'model_status_field':model_status_field,'mrm_model_id': mrm_model_id,'op_api_key': op_api_key,
         'ext_username': username,'ext_api_key': api_key,'input_converter_string': input_converter_string, 'output_converter_string': output_converter_string,
         'input_converter_name': input_converter_name, 'output_converter_name': output_converter_name,
         'op_api_key':op_api_key,'mrm_model_id':mrm_model_id,'model_status_field':model_status_field,
         'openpages_url': openpages_url}




In [ ]:
def wrapper_func(params=params):
    import requests
    import random
    import json
    from urllib.parse import urlparse
    
    cpdhost = params['cpd_host']
    cpd_username = params['cpd_username']
    cpd_api_key = params['cpd_api_key']
    cpd_host = params['cpd_host']
    ext_username = params['ext_username']
    ext_api_key  = params['ext_api_key']
    op_api_key  = params['op_api_key']


    def get_model_use_status(params=params):#always two params in dict format
        data = {
                "statement": "SELECT * FROM [Model] where [Model].[Name]={0}".format(
                    '\u0027' + params['mrm_model_id'] + '\u0027'),
                "skipCount": 0
            }  
        r = requests.post("{0}/grc/api/query".format(params['openpages_url']),
                              headers={"Authorization": "Basic " + params['op_api_key']}, json=data, verify=False)
        data = r.json()
        rows = data['rows']
        model_status_key = params['model_status_field'] #something like 'MRG-Model:Status'
        res = filter(lambda x: x['name'] == model_status_key, rows[0]['fields']['field'])
        model_use_status = list(res)[0]['enumValue']['name']
        return model_use_status
    
    def getToken(cpdhost,username,apikey):
        auth_url = "{0}/icp4d-api/v1/authorize".format(cpdhost)
        auth_response = requests.post(
            auth_url,
            headers={
                "Content-Type": "application/json"
            },
            data='{"username": "'+username+'", "api_key": "'+apikey+'"}',
            verify=False)

        return auth_response.json()['token']
    

    
    def getPredictionExt(params,payload_scoring):
        endpoint=params['ml_endpoint']
        input_converter_string = params["input_converter_string"]
        output_converter_string = params["output_converter_string"]
        input_converter_name = params["input_converter_name"]
        output_converter_name = params["output_converter_name"]
        
        exec(input_converter_string, globals())
        exec(output_converter_string, globals())
        
        localparm = {'model_payload': payload_scoring, 'input_converter_name': input_converter_name}
        input_str = input_converter_name + "(model_payload)"
        
        converted_payload = eval(input_str, globals(), localparm)
        
        header = {'Content-Type': 'application/json'}
        r = requests.post(endpoint, headers=header,data=json.dumps(converted_payload),auth=(ext_username, ext_api_key), verify=False)
        output_resp = r.json()
        localparmout = {'output_resp': output_resp, 'output_converter_name': output_converter_name}
        output_str = output_converter_name + "(output_resp)"
        wml_output = eval(output_str, globals(), localparmout)  
        
        return wml_output
    
    
    
    def score(payload):
        #prediction = getPredictionWML(params['ml_endpoint'],payload)
        if get_model_use_status(params)=="Allow":  
            prediction=getPredictionExt(params,payload)
        else:
            raise ValueError("Model is not in allow state.")
        return prediction

    return score
    

## Test Locally

In [ ]:
#input_payload = {"input_data":[{"fields":["DAY","DAY_OF_WEEK","ORIGIN_AIRPORT","DESTINATION_AIRPORT","DEPARTURE_DELAY","TAXI_OUT","DISTANCE"],"values":[[11,7,"ABQ","DFW",2,11,570],[11,7,"ABQ","DFW",2,11,570]]}]}

In [ ]:
#wrapper_func()(input_payload)

## Store Function in Deployment Space and Deploy

In [ ]:
from ibm_watson_machine_learning import APIClient 
import os

In [ ]:
token = os.environ['USER_ACCESS_TOKEN']
wml_credentials = {
   "token": token,
   "instance_id" : "wml_local",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "3.5"
}
wml_client = APIClient(wml_credentials)

In [ ]:
DEFAULT_DEPLOYMENT_SPACE=deployment_space

filtered_spaces=[ space['metadata']['id'] for space in wml_client.spaces.get_details()['resources'] if space['entity']['name']== DEFAULT_DEPLOYMENT_SPACE]
if len(filtered_spaces)>0:
    space_id=filtered_spaces[0]
else:
    space_metadata = {
        wml_client.spaces.ConfigurationMetaNames.NAME: DEFAULT_DEPLOYMENT_SPACE,
        wml_client.spaces.ConfigurationMetaNames.DESCRIPTION: DEFAULT_DEPLOYMENT_SPACE,
    }
    space_details=wml_client.spaces.store(meta_props=space_metadata);
    space_id=space_details['metadata']['id']
    
print(space_id)

In [ ]:
wml_client.set.default_space(space_id)

In [ ]:
meta_data = { wml_client.repository.FunctionMetaNames.NAME : function_name,
              wml_client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: wml_client.software_specifications.get_id_by_name('default_py3.7_opence')}
function_details = wml_client.repository.store_function(wrapper_func, meta_data)

In [ ]:
function_details

In [ ]:
metadata = {
        wml_client.deployments.ConfigurationMetaNames.NAME: deployment_name,
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
model_id = function_details["metadata"]["id"]
deployment_details = wml_client.deployments.create(artifact_uid=model_id, meta_props=metadata)